## Implement a CNN for CIFAR-10 in PyTorch

In [2]:
import torch
import torchvision
import torchvision.transforms as v2
import torch.nn as nn
import torch.optim as optim

In [11]:
# Load CIFAR-10 dataset
transforms = v2.Compose([
    v2.ToTensor(),
    v2.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
trainset = torchvision.datasets.CIFAR10(".", train=True, transform=transforms, download=True)
testset = torchvision.datasets.CIFAR10(".", train=False, transform=transforms, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [61]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size)


In [62]:
images, labels = next(iter(train_loader))
print(images.shape)
print(labels.shape)

torch.Size([64, 3, 32, 32])
torch.Size([64])


In [63]:
128*6*6

4608

In [64]:
class CustomCNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(nn.Conv2d(3, 64, kernel_size=5, stride=1),
                                nn.MaxPool2d(2, 2),
                                nn.Conv2d(64, 128, kernel_size=3, stride=1),
                                nn.Flatten(),
                                nn.Linear(128*12*12, 128),
                                nn.ReLU(),
                                nn.Linear(128, 10))
    
    def forward(self, X):
        return self.network(X)

In [65]:
# Initialize the model, loss function, and optimizer
model = CustomCNNModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 10
for epoch in range(epochs):
    for images, labels in train_loader:
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [1/10], Loss: 1.1933
Epoch [2/10], Loss: 0.6256
Epoch [3/10], Loss: 0.8469
Epoch [4/10], Loss: 0.5378
Epoch [5/10], Loss: 0.6243
Epoch [6/10], Loss: 0.6183
Epoch [7/10], Loss: 0.6642
Epoch [8/10], Loss: 0.9486
Epoch [9/10], Loss: 0.3950
Epoch [10/10], Loss: 0.1625


In [75]:
correct = 0
total = 0
with torch.no_grad():
    for im, label in test_loader:
        out = model(im)
        total += len(label)
        _, pred = torch.max(out, dim=1)
        correct += torch.sum(pred==label)      

In [77]:
print(correct/total)

tensor(0.6658)
